<h1 align="center">Knee Bend CV Project</h1>

* install and import the required libraries

* define angle calculation function

* read the video and setup the mediapipe variables

* if angle < 100 count the rep and check if the person can not hold for 8 second, if angle get bigger than 100 display a feed back message

In [ ]:
# !pip install mediapipe opencv-python # install mediapipe 

![mediapipe_pose](https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png)

In [7]:
# Import libraries
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime

In [8]:
# Calculate the angle
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [10]:
cap = cv2.VideoCapture("KneeBendVideo.mp4")

# define variables
counter = 0
stage = None
last_detected = 0

# Setup
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened:
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            
            # Visualize angle
            cv2.putText(image, str(angle), (230,300),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (255, 0, 0), 2, cv2.LINE_AA)
            
            # counting logic
            if angle > 160:
                stage = "down"
            if angle < 100 and stage=="down":
                last_detected = datetime.now()
                stage = "up"
                counter +=1
            # Add a holding timer limit of 8 sec
            if angle > 100 and stage=="up":
                if (datetime.now() - last_detected).total_seconds() < 8:
                    cv2.putText(image, 'Keep your knee bent', (250, 200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2, cv2.LINE_AA)
            
        except:
            pass
        
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
       
    
        cv2.imshow('Knee-Bend', image)
    
        if cv2.waitKey(5) & 0xff == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()